In [1]:
import os
import sys
nb_dir = "../include_files"
if nb_dir not in sys.path:
    sys.path.append(nb_dir)


import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
%matplotlib inline
import h5py as h5
import cosmology

from matplotlib.backends.backend_pdf import PdfPages

plt.style.use("./include_files/marius.mplstyle")
fontSize = 15
lineWidth = 1.5

colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']

import contextlib
@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally: 
        np.set_printoptions(**original)

# Read the input data  
  
This consists of the McConnachie and Venn (2020) Gaia EDR3 proper motions. The paper is https://ui.adsabs.harvard.edu/abs/2020RNAAS...4..229M/abstract and the original method paper (applied to DR2) is https://ui.adsabs.harvard.edu/abs/2020AJ....160..124M/abstract .
  
It does not contain the proper motions for Sag, LMC, and SMC -- these are read from the Gaia DR2 flagship paper on proper motions: https://ui.adsabs.harvard.edu/abs/2018A%26A...616A..12G/abstract  
  
  
### First start by reading the McConnacchie & Venn data.

In [3]:
# read the McConnachie data file
from astropy.io import fits
from astropy import units as u
from astropy.table import Table

fits_data_filename = "data/NearbyGalaxies_Jan2021_PUBLIC.fits"
with fits.open(fits_data_filename) as hdul:
    hdul.info()
    
data = Table.read(fits_data_filename)
# print(data.meta)

# make a copy to keep track off
data3 = data.copy()

data[:3]

Filename: data/NearbyGalaxies_Jan2021_PUBLIC.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (5262,)   uint8   
  1  Joined        1 BinTableHDU    125   144R x 49C   [16A, 10A, 11A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, E, E, E, E, 53A]   


GalaxyName,RA,Dec,EB-V,dmod,dmod+,dmod-,vh,vh+,vh-,Vmag,Vmag+,Vmag-,PA,PA+,PA-,e=1-b/a,e+,e-,muVo,muVo+,muVo-,rh,rh+,rh-,sigma_s,sigma_s+,sigma_s-,vrot_s,vrot_s+,vrot_s-,MHI,sigma_g,sigma_g+,sigma_g-,vrot_g,vrot_g+,vrot_g-,[Fe/H],feh+,feh-,F,pmra,epmra+,epmra-,pmdec,epmdec+,epmdec-,References
bytes16,bytes10,bytes11,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,bytes53
*Bootes3,13:57:12.0,26:48:0.0,0.021,18.35,0.1,0.1,197.5,3.8,3.8,12.6,0.5,0.5,90.0,999.0,999.0,0.5,999.0,999.0,31.3,0.3,0.3,999.0,999.0,999.0,14.0,3.2,3.2,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,-2.1,0.2,0.2,2,--,--,--,--,--,--,(14)(15)(99)
*CanisMajor,7:12:35.0,-27:40:0.0,0.264,14.29,0.3,0.3,87.0,4.0,4.0,-0.1,0.8,0.8,123.0,999.0,999.0,999.0,999.0,999.0,24.0,0.6,0.6,999.0,999.0,999.0,20.0,3.0,3.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,-0.5,0.2,0.2,4,--,--,--,--,--,--,(1)(2)(95)(176)
*Cetus2,1:17:52.8,-17:25:12.0,0.02,17.1,0.1,0.1,999.0,999.0,999.0,17.4,0.7,0.7,999.0,999.0,999.0,999.0,999.0,999.0,28.55,1.2,1.2,1.9,1.0,0.5,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,-1.28,0.07,0.07,5,2.84,0.05,0.07,0.46,0.06,0.07,(232)(260)(266)(267)


### Now read the data from the Gaia DR2. This is in a txt file.

In [4]:
data2_pm = np.loadtxt( "data/MW_sats_McConnachie_2020_Gaia_DR2.txt", usecols=[8,9,10,11] )[:11]
data2_satName = np.loadtxt( "data/MW_sats_McConnachie_2020_Gaia_DR2.txt", usecols=[2] )[:11]

### Copy the proper motions for Sag, LMC, and SMC

In [10]:
# Find the satellites in question: Sag, LMC and SMC in the table
target_sats = [ 'SagittariusdSph', 'LMC', 'SMC']
numSats_total = len(data['GalaxyName'])
numSats = len(target_sats)

sel_target = np.array( [ data['GalaxyName'][i].strip() in target_sats for i in range(numSats_total) ] )
print( "Found {0} satellites out of {1} searched for.\n\t Found all: {2}".format( sel_target.sum(), numSats, sel_target.sum()==numSats) )

# copy the proper motion data
reorder = (data['dmod'][sel_target]).argsort() # order the sats using the distance modulus
index = np.arange( numSats_total )[sel_target]

data['pmra'][index[:3]] = data2_pm[:3,0]
data['epmra+'][index[:3]] = data2_pm[:3,1]
data['epmra-'][index[:3]] = data2_pm[:3,1]

data['pmdec'][index[:3]] = data2_pm[:3,2]
data['epmdec+'][index[:3]] = data2_pm[:3,3]
data['epmdec-'][index[:3]] = data2_pm[:3,3]

print( "Updated proper motion data:" )
data[['GalaxyName','pmra','epmra+','epmra-','pmdec','epmdec+','epmdec-']][sel_target]

Found 3 satellites out of 3 searched for.
	 Found all: True
Updated proper motion data:


GalaxyName,pmra,epmra+,epmra-,pmdec,epmdec+,epmdec-
bytes16,float32,float32,float32,float32,float32,float32
LMC,-2.692,0.001,0.001,-1.359,0.001,0.001
SMC,1.85,0.03,0.03,0.234,0.03,0.03
SagittariusdSph,0.797,0.03,0.03,-1.22,0.03,0.03


# Identify satellites within 300 kpc from the Galactic Centre  
  
Valid satellites should be within 300 kpc from the Galactic Centre and have distance, radial velocity and proper motion measurements. We select satellites using the most likely distance modulus.

In [34]:
import astropy.coordinates as coord

# define the selection criteria
dist_max = 300 * u.kpc
dmod_max = coord.Distance( value=dist_max ).distmod  # transform to distance modulus
vrad_max = 900. * u.km / u.s # larger values indicate a missing meaurement 

# select by distance
sel_distance = data['dmod'] <= dmod_max.value
print( "Satellites that passed the distance selection criterion: ", sel_distance.sum() )

# select by radial velocity
sel_vrad = sel_distance * (data['vh'] <= vrad_max.value)
print( "       and radial velocity criterion: ", sel_vrad.sum() )

# select by proper motions
sel_pm  = np.array( [ np.isscalar(data['pmra'][i]) & np.isscalar(data['pmdec'][i]) for i in range(numSats_total) ] )
sel_valid = sel_vrad * sel_pm
print( "       and proper motion criterion: ", sel_valid.sum() )

data[['GalaxyName','dmod','vh','pmra','pmdec',]][sel_valid]


Satellites that passed the distance selection criterion:  62
       and radial velocity criterion:  51
       and proper motion criterion:  48


GalaxyName,dmod,vh,pmra,pmdec
bytes16,float32,float32,float32,float32
*Columba1,21.3,153.7,0.19,-0.36
*Draco2,16.67,-342.5,1.08,0.91
*Grus1,20.4,-140.5,0.07,-0.29
*Grus2,18.62,-110.0,0.38,-1.46
*Horologium1,19.5,112.8,0.82,-0.61
*Horologium2,19.46,168.7,0.76,-0.41
*Pegasus3,21.56,-222.9,0.06,-0.2
*Phoenix2,19.6,32.4,0.48,-1.17
*Reticulum2,17.4,64.7,2.39,-1.36


# Transform from Solar to Galactocentric coordinates  
  
https://docs.astropy.org/en/stable/api/astropy.coordinates.Galactocentric.html#astropy.coordinates.Galactocentric  

To account for errors, generate many Monte Carlo samples in observed space taking into account the measurement errors in observed satellite positions and also in the position and velocity of the Sun. The Solar reference frame values are taken from:  
- Sun's position in the disc plane $R_\odot= (8.178 \pm 0.022) ~\rm{kpc}$ from [Gravity Collaboration et al (2019)](https://ui.adsabs.harvard.edu/abs/2019A%26A...625L..10G/abstract)  
- Sun's vertical position (i.e. from disc plane) $z=(0.020\pm0.005) ~\rm{kpc}$ from [Joshi (2007)](https://ui.adsabs.harvard.edu/abs/2007MNRAS.378..768J/exportcitation)  
- circular velocity at the Sun's position $V_{\rm circ} = (234.7\pm1.7)~\rm{km/s}$ from [Nitschai et al (2021)](https://ui.adsabs.harvard.edu/abs/2021ApJ...916..112N/abstract)  
- Sun's motion with respect to the LSR (local standard of rest) $(U,V,W)=(11.10 \pm 0.72, 12.24\pm0.47, 7.25\pm0.37)~\rm{km/s}$ from [Schonrich, Binney and Dehnen (2010)](https://ui.adsabs.harvard.edu/abs/2010MNRAS.403.1829S/abstract)


### Function for doing the transformation using *astropy*

In [36]:
# transform to Galactocentric coordinates
import astropy.coordinates as coord

def satellite_phase_space_wrt_GC( ra, dec, dist, rad_vel, pmra, pmdec, Sun_R, Sun_z, Sun_vel ):
    gc_frame = coord.Galactocentric( galcen_distance=Sun_R*u.kpc, galcen_v_sun=Sun_vel*u.km/u.s, z_sun=Sun_z*u.kpc)
    
    coord_Sun = coord.SkyCoord( ra=ra, dec=dec, unit=(u.hourangle, u.deg), frame='icrs',
                        distance=dist*u.kpc, 
                        pm_ra_cosdec=pmra*u.mas/u.yr,
                        pm_dec=pmdec*u.mas/u.yr,
                        radial_velocity=rad_vel*u.km/u.s )
    coord_GC = coord_Sun.transform_to(gc_frame)
    return coord_GC


### The measurements and their errors

In [33]:
sun_vCirc = np.array( [234.7, 1.7] )   # Vcirc at Sun's position: value + error from Nitschai + 2021 -- https://ui.adsabs.harvard.edu/abs/2021ApJ...916..112N/abstract
sun_R = np.array( [8.178, 0.022] )  # in-plane distance of the Sun from GC: value + error from Gravity Collaboration + 2019 -- https://ui.adsabs.harvard.edu/abs/2019A%26A...625L..10G/abstract
sun_LSR   = np.array( [ [11.10, 0.72], [12.24,0.47], [7.25,0.37] ]  )  # (U,V,W) vel. of sun w.r.t. local standard of rest - Schonrich, Binney and Dehnen 2010 -- https://ui.adsabs.harvard.edu/abs/2010MNRAS.403.1829S/abstract
sun_z = np.array( [0.020,0.01])   # Sun's z displacement Joshi (2007) -- https://ui.adsabs.harvard.edu/abs/2007MNRAS.378..768J/exportcitation

### First transform the most likely (ML) measurements to sort the satellites from closest to farthest from the GC

In [38]:
sat_dist= coord.Distance( distmod=data['dmod'][sel_valid], unit=u.kpc).value
sun_vel = [sun_LSR[0,0], sun_vCirc[0]+sun_LSR[1,0], sun_LSR[2,0]]

coords = satellite_phase_space_wrt_GC( ra=data['RA'][sel_valid], dec=data['Dec'][sel_valid], \
                                          dist=sat_dist, rad_vel=data['vh'][sel_valid], \
                                          pmra=data['pmra'][sel_valid], pmdec =data['pmdec'][sel_valid], \
                                          Sun_R=sun_R[0], Sun_z=sun_z[0], Sun_vel=sun_vel )

# sort satellites in order of distance from GC
pos_GC_ML = np.column_stack( (coords.x.value, coords.y.value, coords.z.value) )   # most likely value
vel_GC_ML = np.column_stack( (coords.v_x.value, coords.v_y.value, coords.v_z.value) )   # most likely value
dis_GC_ML = np.sqrt( (pos_GC_ML**2).sum(axis=1) )
order = dis_GC_ML.argsort()

sat_names = np.array( [name.strip() for name in data['GalaxyName'][sel_valid]] )[order]
pos_GC_ML = pos_GC_ML[order]
vel_GC_ML = vel_GC_ML[order]
dis_GC_ML = dis_GC_ML[order]

print( "The most likely distance of the satellites from the Galactic Centre [kpc]:\n", dis_GC_ML )

The most likely distance of the satellites from the Galactic Centre [kpc]:
 [ 18.53525784  23.24190385  23.86991506  25.67585236  27.80758119
  28.97436221  31.61511777  36.46303968  37.05714199  37.82876922
  39.50173714  40.65990978  42.7721595   44.83058174  45.5181823
  48.4209829   49.99275326  52.07513176  53.7577618   61.26869187
  63.98948074  66.10985892  75.87278278  77.7729385   79.03238974
  79.73888121  80.20551663  85.99698782  88.93724456  91.60032434
 101.51526314 105.11257784 106.8018824  116.41242652 116.43536033
 126.22327731 131.44799103 132.78946189 149.27548209 154.74458046
 160.57914828 181.94853044 186.54232463 196.6577967  203.17046842
 217.53369461 235.89375596 257.33570267]


### Code for generating the Monte Carlo samples

In [40]:
# loop over the number of MC realizations and for each one obtain a set of positions and velocities w.r.t. the Galactic Centre
number_MC = 1000  # number of Monte Carlo realizations
numSats = sel_valid.sum()

# arrays for storing the final satellite positions and velocities in GC reference frame
pos_GC = np.empty( (number_MC,numSats,3), np.float64 )
vel_GC = np.empty( (number_MC,numSats,3), np.float64 )


# generate MC samples for the satellite observables
sat_dist = np.empty( (number_MC,numSats), np.float64 )
sat_radV = np.empty( (number_MC,numSats), np.float64 )
sat_pmra = np.empty( (number_MC,numSats), np.float64 )
sat_pmdec= np.empty( (number_MC,numSats), np.float64 )
sat_index= np.arange(numSats_total)[sel_valid]
for i in range(numSats):
    index = sat_index[i]
    
    edmod = 0.5 * (data['dmod+'][index] + data['dmod-'][index])
    dist_mod = np.random.normal( loc=data['dmod'][index], scale=edmod, size=number_MC )
    sat_dist[:,i] = coord.Distance( distmod=dist_mod, unit=u.kpc).value
    
    evh = 0.5 * (data['vh+'][index] + data['vh-'][index])
    sat_radV[:,i] = np.random.normal( loc=data['vh'][index], scale=evh, size=number_MC )
    
    epmra = 0.5 * (data['epmra+'][index] + data['epmra-'][index])
    sat_pmra[:,i] = np.random.normal( loc=data['pmra'][index], scale=epmra, size=number_MC )
    
    epmdec = 0.5 * (data['epmdec+'][index] + data['epmdec-'][index])
    sat_pmdec[:,i] = np.random.normal( loc=data['pmdec'][index], scale=epmdec, size=number_MC )


# generate MC samples for the Sun's position and velocity
Sun_R = np.random.normal( loc=sun_R[0], scale=sun_R[1], size=number_MC )
Sun_z = np.random.normal( loc=sun_z[0], scale=sun_z[1], size=number_MC )
Sun_U = np.random.normal( loc=sun_LSR[0,0], scale=sun_LSR[0,1], size=number_MC )
Sun_V = np.random.normal( loc=sun_LSR[1,0], scale=sun_LSR[1,1], size=number_MC ) + np.random.normal( loc=sun_vCirc[0], scale=sun_vCirc[1], size=number_MC )
Sun_W = np.random.normal( loc=sun_LSR[2,0], scale=sun_LSR[2,1], size=number_MC )
Sun_vel = np.column_stack( (Sun_U,Sun_V,Sun_W) )


# transform from solar to galactic coordinates
for i in range(number_MC):
    coords = satellite_phase_space_wrt_GC( ra=data['RA'][sel_valid], dec=data['Dec'][sel_valid], \
                                          dist=sat_dist[i], rad_vel=sat_radV[i], \
                                          pmra=sat_pmra[i], pmdec =sat_pmdec[i], \
                                          Sun_R=Sun_R[i], Sun_z=Sun_z[i], Sun_vel=Sun_vel[i] )
    pos_GC[i,] = np.column_stack( (coords.x.value, coords.y.value, coords.z.value) )[order]
    vel_GC[i,] = np.column_stack( (coords.v_x.value, coords.v_y.value, coords.v_z.value) )[order]
    

### Save the MC samples

In [41]:
# output the satellite data to an hdf5 file
outfile = "data_output/MW_sats_MC_samples.npz"
np.savez( outfile, satellite_names=sat_names, positions_GC=pos_GC, velocities_GC=vel_GC, \
         positions_GC_ML=pos_GC_ML, velocities_GC_ML=vel_GC_ML, distances_GC_ML=dis_GC_ML )

import h5py
outfile = "data_output/MW_sats_MC_samples.hdf5"
with h5py.File( outfile, 'w' ) as hf:
    hf.create_dataset( 'satellite_names'  , data=np.array(sat_names,dtype='S') )
    
    # write the data of hosts with LMC
    g1 = hf.create_group( "Monte_Carlo_realizations" )
    g1.create_dataset( 'MC_pos'  , data=pos_GC, dtype='f8' )
    g1.create_dataset( 'MC_vel'  , data=vel_GC, dtype='f8' )
    
    g1 = hf.create_group( "most_likely_values" )
    g1.create_dataset( 'pos'  , data=pos_GC_ML, dtype='f8' )
    g1.create_dataset( 'vel'  , data=vel_GC_ML, dtype='f8' )
    g1.create_dataset( 'dis'  , data=dis_GC_ML, dtype='f8' )
    
    # write the data of satellites around the desired hosts
    g2 = hf.create_group( "Observable_data" )
    g2.create_dataset( 'sat_dist'  , data=sat_dist, dtype='f4' )
    g2.create_dataset( 'sat_radV'  , data=sat_radV, dtype='f4' )
    g2.create_dataset( 'sat_pmra'  , data=sat_pmra, dtype='f4' )
    g2.create_dataset( 'sat_pmdec' , data=sat_pmdec, dtype='f4' )
    g2.create_dataset( 'Sun_R'   , data=Sun_R, dtype='f4' )
    g2.create_dataset( 'Sun_z'   , data=Sun_z, dtype='f4' )
    g2.create_dataset( 'Sun_vel' , data=Sun_vel, dtype='f4' )
    
    grp = hf.create_group( "metadata" )
    grp.attrs["units"]     = np.string_( "Positions: kpc;  Velocities: km/s" )
    grp.attrs["num_Monte_Carlos"] = pos_GC.shape[0]
    grp.attrs["num_satellites"]   = pos_GC.shape[1]
    grp.attrs["program_options"]  = "Obtained using the 'MW_satellites_errors.ipynb' jupyter notebook"

### A quick check of the position and velocity percentage error

In [42]:
velM_ML = (vel_GC_ML**2).sum(axis=1)**0.5

out = np.column_stack( (pos_GC.std(axis=0)/dis_GC_ML[:,None], vel_GC.std(axis=0)/velM_ML[:,None]) ) * 100.

print("Uncertainties in the coordinates and velocities of the satellites (in percentage):")
print("                  x     y     z     v_x   v_y   v_z")
for i in range( len(sat_names) ):
    print( "%15s   " % sat_names[i], end="" )
    for j in range(6):
        print("%.2f  " % out[i,j], end="")
    print()
# print( np.column_stack( (sat_names, pos_GC.std(axis=0)/dis_GC_ML[:,None], vel_GC.std(axis=0)/velM_ML[:,None]) ) )


Uncertainties in the coordinates and velocities of the satellites (in percentage):
                  x     y     z     v_x   v_y   v_z
SagittariusdSph   9.68  0.94  2.48  1.16  3.20  4.24  
       *Tucana3   3.22  3.19  6.78  2.11  4.95  3.37  
        *Draco2   0.25  1.50  1.41  2.37  1.68  1.76  
        Hydrus1   0.72  1.36  1.14  1.03  1.87  1.58  
       Segue(1)   3.75  3.22  6.00  3.04  13.39  8.58  
        Carina3   0.09  4.24  1.29  2.09  1.83  5.34  
    *Reticulum2   0.41  5.87  6.95  1.27  12.37  10.38  
   *Triangulum2   2.82  2.28  1.59  1.97  2.45  2.74  
        Carina2   0.07  2.14  0.66  1.17  0.84  1.96  
     UrsaMajor2   7.87  4.11  6.83  3.90  14.33  12.90  
        Bootes2   1.04  0.11  2.68  4.86  5.00  2.32  
         Segue2   2.68  1.58  2.43  6.12  7.55  4.04  
       Willman1   8.69  3.42  14.33  11.31  29.06  12.74  
  ComaBerenices   0.46  0.87  8.81  8.54  8.85  0.60  
       *Tucana4   3.64  3.87  7.69  6.18  16.30  8.57  
         *Grus2   6.39  1.00  

# Generate only the 68 percentile MC samples  
  
Do so by generating MC samples for each satellite and keeping only the 68% with the highest likelihood for each satellite.

In [151]:
# function to generate MC and calculate the likelihood for each variable
def MC_samples_and_loglikelihood(N,mean,sigma):
    _samples = np.random.normal( loc=mean, scale=sigma, size=N )
    _loglikelihood = -0.5 * (_samples-mean)**2 / sigma**2
    return _samples, _loglikelihood


# function to generate the required percentile MC samples
def generate_MC_percentiles_satellites(percentile,N,index):
    N2 = int(N / (percentile/100)) # to obtain N samples that correspond to the 'percentile' percentage of the distribution we need to generate 'N2' samples
    
    _edmod = 0.5 * (data['dmod+'][index] + data['dmod-'][index])
    _dist_mod, _logL_dist = MC_samples_and_loglikelihood( N2, data['dmod'][index], _edmod )
    _sat_dist = coord.Distance( distmod=_dist_mod, unit=u.kpc).value
    
    _evh = 0.5 * (data['vh+'][index] + data['vh-'][index])
    _sat_radV, _logL_radV = MC_samples_and_loglikelihood( N2, data['vh'][index], _evh )
    
    _epmra = 0.5 * (data['epmra+'][index] + data['epmra-'][index])
    _sat_pmra, _logL_pmra = MC_samples_and_loglikelihood( N2, data['pmra'][index], _epmra )
    
    _epmdec = 0.5 * (data['epmdec+'][index] + data['epmdec-'][index])
    _sat_pmdec, _logL_pmdec = MC_samples_and_loglikelihood( N2, data['pmdec'][index], _epmdec )
    
    # calculate the total log likelihood and select the N highest
    _logL = _logL_dist + _logL_radV + _logL_pmra + _logL_pmdec
    _order = _logL.argsort()[::-1][:N]
    _sel = np.zeros( N2, bool )
    _sel[_order] = True
    
    return _sat_dist[_sel], _sat_radV[_sel], _sat_pmra[_sel], _sat_pmdec[_sel]


def generate_MC_percentiles_Sun(percentile,N):
    N2 = int(N / (percentile/100)) # to obtain N samples that correspond to the 'percentile' percentage of the distribution we need to generate 'N2' samples
    
    _Sun_R, _logL_R = MC_samples_and_loglikelihood( N2, sun_R[0], sun_R[1] )
    _Sun_z, _logL_z = MC_samples_and_loglikelihood( N2, sun_z[0], sun_z[1] )
    _Sun_Vc, _logL_Vc = MC_samples_and_loglikelihood( N2, sun_vCirc[0], sun_vCirc[1] )
    _Sun_U, _logL_U = MC_samples_and_loglikelihood( N2, sun_LSR[0,0], sun_LSR[0,1] )
    _Sun_V, _logL_V = MC_samples_and_loglikelihood( N2, sun_LSR[1,0], sun_LSR[1,1] ) 
    _Sun_W, _logL_W = MC_samples_and_loglikelihood( N2, sun_LSR[2,0], sun_LSR[2,1] )
    _Sun_vel = np.column_stack( (_Sun_U,_Sun_V+_Sun_Vc,_Sun_W) )
    
    # calculate the total log likelihood and select the N highest
    _logL = _logL_R + _logL_z + _logL_Vc + _logL_U + _logL_V + _logL_W
    _order = _logL.argsort()[::-1][:N]
    _sel = np.zeros( N2, bool )
    _sel[_order] = True
    
    return _Sun_R[_sel], _Sun_z[_sel], _Sun_vel[_sel]
    
# generate_MC_percentiles_satellites(68,10,sat_index[0])
generate_MC_percentiles_Sun(68,10)

(array([8.1667253 , 8.17881634, 8.18006186, 8.14363859, 8.16244625,
        8.16271786, 8.19193653, 8.17399763, 8.18411458, 8.19750135]),
 array([0.02781304, 0.0240489 , 0.0229561 , 0.01262945, 0.02470571,
        0.01242147, 0.01314364, 0.02039701, 0.01321091, 0.02530495]),
 array([[ 11.63646166, 244.52941432,   7.5069091 ],
        [ 11.19297362, 246.19500036,   7.48087737],
        [ 11.28344146, 246.2239645 ,   6.86115774],
        [ 11.60690759, 248.33495732,   7.16981539],
        [ 12.41502673, 248.41146003,   7.1732111 ],
        [ 11.02333872, 248.38471721,   7.41756535],
        [ 10.95210486, 245.08689365,   7.34562999],
        [ 10.29761859, 248.70344851,   7.16713902],
        [ 10.81020644, 245.15546113,   7.33105443],
        [ 10.7565499 , 246.10269224,   7.81158857]]))

### Now generate the 68 percentile MC samples

In [155]:
# loop over the number of MC realizations and for each one obtain a set of positions and velocities w.r.t. the Galactic Centre
number_MC = 1000  # number of Monte Carlo realizations
target_percetile = 68

# arrays for storing the final satellite positions and velocities in GC reference frame
pos_GC = np.empty( (number_MC,numSats,3), np.float64 )
vel_GC = np.empty( (number_MC,numSats,3), np.float64 )


# generate MC samples for the satellite observables
sat_dist = np.empty( (number_MC,numSats), np.float64 )
sat_radV = np.empty( (number_MC,numSats), np.float64 )
sat_pmra = np.empty( (number_MC,numSats), np.float64 )
sat_pmdec= np.empty( (number_MC,numSats), np.float64 )
sat_index= np.arange(numSats_total)[sel_valid]
for i in range(numSats):
    sat_dist[:,i], sat_radV[:,i], sat_pmra[:,i], sat_pmdec[:,i] = generate_MC_percentiles_satellites( target_percetile, number_MC, sat_index[i] )

# generate MC samples for the Sun's position and velocity
Sun_R, Sun_z, Sun_vel = generate_MC_percentiles_Sun( target_percetile, number_MC )


# transform from solar to galactic coordinates
for i in range(number_MC):
    coords = satellite_phase_space_wrt_GC( ra=data['RA'][sel_valid], dec=data['Dec'][sel_valid], \
                                          dist=sat_dist[i], rad_vel=sat_radV[i], \
                                          pmra=sat_pmra[i], pmdec =sat_pmdec[i], \
                                          Sun_R=Sun_R[i], Sun_z=Sun_z[i], Sun_vel=Sun_vel[i] )
    pos_GC[i,] = np.column_stack( (coords.x.value, coords.y.value, coords.z.value) )[order]
    vel_GC[i,] = np.column_stack( (coords.v_x.value, coords.v_y.value, coords.v_z.value) )[order]

### Save the MC samples

In [156]:
# output the satellite data to an hdf5 file
outfile = "data_output/MW_sats_MC_samples_68per.npz"
np.savez( outfile, satellite_names=sat_names, positions_GC=pos_GC, velocities_GC=vel_GC, \
         positions_GC_ML=pos_GC_ML, velocities_GC_ML=vel_GC_ML, distances_GC_ML=dis_GC_ML )

import h5py
outfile = "data_output/MW_sats_MC_samples_68per.hdf5"
with h5py.File( outfile, 'w' ) as hf:
    hf.create_dataset( 'satellite_names'  , data=np.array(sat_names,dtype='S') )
    
    # write the data of hosts with LMC
    g1 = hf.create_group( "MC_samples" )
    g1.create_dataset( 'pos'  , data=pos_GC, dtype='f8' )
    g1.create_dataset( 'vel'  , data=vel_GC, dtype='f8' )
    
    g1 = hf.create_group( "most_likely_values" )
    g1.create_dataset( 'pos'  , data=pos_GC_ML, dtype='f8' )
    g1.create_dataset( 'vel'  , data=vel_GC_ML, dtype='f8' )
    g1.create_dataset( 'dis'  , data=dis_GC_ML, dtype='f8' )
    
    # write the data of satellites around the desired hosts
    g2 = hf.create_group( "Observable_data" )
    g2.create_dataset( 'sat_dist'  , data=sat_dist, dtype='f4' )
    g2.create_dataset( 'sat_radV'  , data=sat_radV, dtype='f4' )
    g2.create_dataset( 'sat_pmra'  , data=sat_pmra, dtype='f4' )
    g2.create_dataset( 'sat_pmdec' , data=sat_pmdec, dtype='f4' )
    g2.create_dataset( 'Sun_R'   , data=Sun_R, dtype='f4' )
    g2.create_dataset( 'Sun_z'   , data=Sun_z, dtype='f4' )
    g2.create_dataset( 'Sun_vel' , data=Sun_vel, dtype='f4' )
    
    grp = hf.create_group( "metadata" )
    grp.attrs["units"]     = np.string_( "Positions: kpc;  Velocities: km/s" )
    grp.attrs["num_Monte_Carlos"] = pos_GC.shape[0]
    grp.attrs["num_satellites"]   = pos_GC.shape[1]
    grp.attrs["program_options"]  = "Obtained using the 'MW_satellites_errors.ipynb' jupyter notebook"

### A quick check of the position and velocity percentage error

In [157]:
velM_ML = (vel_GC_ML**2).sum(axis=1)**0.5

out = np.column_stack( (pos_GC.std(axis=0)/dis_GC_ML[:,None], vel_GC.std(axis=0)/velM_ML[:,None]) ) * 100.

print("Uncertainties in the coordinates and velocities of the satellites (in percentage):")
print("                  x     y     z     v_x   v_y   v_z")
for i in range( len(sat_names) ):
    print( "%15s   " % sat_names[i], end="" )
    for j in range(6):
        print("%.2f  " % out[i,j], end="")
    print()
# print( np.column_stack( (sat_names, pos_GC.std(axis=0)/dis_GC_ML[:,None], vel_GC.std(axis=0)/velM_ML[:,None]) ) )


Uncertainties in the coordinates and velocities of the satellites (in percentage):
                  x     y     z     v_x   v_y   v_z
SagittariusdSph   0.39  0.04  0.13  0.51  0.50  0.27  
            LMC   0.16  0.80  0.53  0.37  1.13  1.18  
            SMC   0.41  0.63  0.73  0.37  1.53  1.07  
      UrsaMinor   0.14  0.47  0.48  0.40  0.84  0.19  
          Draco   0.06  0.30  0.21  0.37  0.79  0.18  
       Sculptor   0.11  0.16  1.36  0.32  0.93  0.18  
     Sextans(1)   0.13  0.23  0.23  0.33  0.63  0.21  
         Carina   0.64  3.63  1.50  0.34  2.41  5.45  
         Fornax   0.26  0.37  0.98  0.48  2.90  1.03  
           Leo2   1.17  1.00  3.70  2.82  6.04  2.48  
           Leo1   1.11  1.15  1.84  0.34  1.88  1.03  


19.3537747244507 mag
[ 0.          0.00292218 -0.00292612] mag
